## Locality Sensitive Hashing

In [11]:
import pickle
from pathlib import Path
import numpy as np
import pandas as pd
import random
from itertools import combinations
root = Path(".")

In [12]:
article_1 = pd.read_csv('articles1.csv')

In [13]:
lsh_stats = {
    'b' : list(),
    'r' : list(),
    'Number of hash functions' : list(),
    'True Positive' : list(),
    'True Negative' : list(),
    'False Positive' : list(),
    'False Negative' : list(),
    'Candidate Pair count' : list()
}

We first load the pickled documents into RAM. The first step of LSH is the creation of the signature matrix. Since incident matrix is too sparse and dimension heavy, we create a compact signature matrix using the minhashing technique. 
</br> </br>
<span style="color:yellow">get_hash_functions(hsh_cnt, mod)</span> generates hsh_cnt random hash functions which will be used for minhashing. The generated hash functions are of the form $(ax + b)$ % $mod$
</br> </br>
MinHashing Technique : 
</br>
The signature matrix is created using the minhash technique where we apply all of the hash functions row-wise and update the minimum hash value in each document which has the current shingle.
</br></br>
The general idea of LSH is to find a algorithm such that if we input signatures of 2 documents, it tells us that those 2 documents form a candidate pair. The signature matrix is divided into b bands of r row each, for each band we hash its portion of the column in the corresponding bucket, Candidate column pairs are those that hash to the same bucket for at least 1 band. The results are verified by actually checking if those documents actually have similarity >= threshold. The top 3 similar documents, number of candidate pairs along with statistics depicting performance of LSH (True Positive, True Negative, False Positive, False Negative) is returned.

In [14]:
class LSH:
    def __init__(self, b, r, hash_cnt) -> None:
        self.unpickle()
        self.hash_functions = self.get_hash_functions(hash_cnt, len(self.incident_matrix))
        self.min_hash()
        self.make_candidate_pairs(b, r)

    def get_hash_functions(self, hsh_cnt, mod): 
        hash_function = list()
        for _ in range(hsh_cnt) : 
            a = random.randint(1, 100)
            b = random.randint(1, 100)
            hash_function.append((a, b, mod))
        return hash_function

    def min_hash(self):
        self.signature_matrix = np.zeros(shape=(len(self.hash_functions), len(self.documents)))
        self.signature_matrix.fill(10 ** 18)

        for shingle_row in range(len(self.incident_matrix)): 
            cur_hsh = 0
            shingle_row_val = np.array(self.incident_matrix[shingle_row])
            ids_take = np.nonzero(shingle_row_val)
            
            for (a, b, mod) in self.hash_functions: 
                hsh_value = ((shingle_row + 1) * a + b) % mod

                for doc_id in ids_take[0]:
                    if shingle_row_val[doc_id] == 1:
                        self.signature_matrix[cur_hsh][doc_id] = min(self.signature_matrix[cur_hsh][doc_id], hsh_value)
                cur_hsh += 1

        my_path = root / "Pickled_files" / "Signature_Matrix"
        dbfile = open(my_path, 'wb')
        pickle.dump(self.signature_matrix, dbfile) 
        dbfile.close()

    def make_candidate_pairs(self, b, r): 
        self.candidate_pairs = set()
        self.taken_candidate_pairs = np.zeros(shape=(len(self.documents), len(self.documents)))
        cur_band_start, cur_band_end = 0, r - 1
        for _ in range(b):
            band_signature = dict() 
            for doc_id in range(len(self.documents)): 
                matrix = tuple(self.signature_matrix[cur_band_start : cur_band_end + 1, doc_id])
                if band_signature.get(matrix) == None: 
                    band_signature[matrix] = [doc_id]
                else: 
                    band_signature[matrix].append(doc_id)
            for docs in band_signature.items(): 
                if len(docs[1]) > 1: 
                    c_pairs = list(combinations(docs[1], 2))
                    for (a, b) in c_pairs: 
                        self.candidate_pairs.add((min(a, b), max(a, b)))
                        self.taken_candidate_pairs[a][b] = self.taken_candidate_pairs[b][a] = 1
            cur_band_end += r
            cur_band_start += r

    def jaccard_similarity(self, a, b) :
        bits = np.bitwise_and(a, b)
        dif = np.bitwise_xor(a, b)
        return float(np.count_nonzero(bits == 1) / (np.count_nonzero(dif == 1) + np.count_nonzero(bits == 1))) 
    
    def get_similar_documents(self, threshold): 
        similar_docs = list()
        self.similarity_matrix = np.zeros(shape=(len(self.documents), len(self.documents)))
        sims = list()

        matrixes = [None for _ in range(len(self.documents))]
        for i in range(len(self.documents)):
            matrixes[i] = self.incident_matrix[:, i]
            matrixes[i] = matrixes[i].astype(int)

        for doc1 in range(len(self.documents)): 
            matrix1 = matrixes[doc1]

            for doc2 in range(doc1 + 1, len(self.documents)): 
                matrix2 = matrixes[doc2]
            
                similarity = self.jaccard_similarity(matrix1, matrix2)
                sims.append(similarity)
                self.similarity_matrix[doc1][doc2] = self.similarity_matrix[doc2][doc1] = similarity
                if similarity >= threshold: 
                    similar_docs.append((doc1, doc2, similarity))
        return list(set(similar_docs)), sims
    
    def process_candidate_pairs(self, threshold):
        false_positive = false_negative = 0
        true_positive = true_negative = 0

        for doc1 in range(len(self.documents)):
            for doc2 in range(doc1 + 1, len(self.documents)):
                if self.taken_candidate_pairs[doc1][doc2] == 1:
                    if self.similarity_matrix[doc1][doc2] >= threshold:
                        true_positive += 1
                    else:
                        false_positive += 1
                elif self.similarity_matrix[doc1][doc2] >= threshold:
                    false_negative += 1
                else:
                    true_negative += 1

        print(f'''
            true positive count = {true_positive}
            true negative count = {true_negative}
            false positive count = {false_positive}
            false negative count = {false_negative}
        ''')
        return true_positive, true_negative, false_positive, false_negative 

    def unpickle(self):
        my_path = root / "Pickled_files" / "Incident_Matrix"
        dbfile = open(my_path, 'rb')     
        self.incident_matrix = pickle.load(dbfile)
        dbfile.close()

        my_path = root / "Pickled_files" / "Shingle_id"
        dbfile = open(my_path, 'rb')
        self.shingle_id = pickle.load(dbfile)
        dbfile.close()

        my_path = root / "Pickled_files" / "Shingles"
        dbfile = open(my_path, 'rb')
        self.shingles = pickle.load(dbfile)
        dbfile.close()

        my_path = root / "Pickled_files" / "documents"
        dbfile = open(my_path, 'rb')
        self.documents = pickle.load(dbfile)
        dbfile.close()

In [15]:
def print_docs(similar_docs):
    for x in similar_docs:
        print('##############################################################')
        print(f'Doc1 is {article_1.title[x[0]]}')
        print(f'Doc2 is {article_1.title[x[1]]}')
        print(f'Similarity is {x[2] * 100}%')

### r = 5, b = 10, hash count = 50

In [16]:
lsh = LSH(b=10, r=5, hash_cnt=50)
similar_docs, sims = lsh.get_similar_documents(0.6)
similar_docs.sort(key=lambda x: -x[2])
print_docs(similar_docs[:3])
true_positive, true_negative, false_positive, false_negative = lsh.process_candidate_pairs(0.6)
lsh_stats['b'].append(10)
lsh_stats['r'].append(5)
lsh_stats['Number of hash functions'].append(50)
lsh_stats['True Positive'].append(true_positive)
lsh_stats['True Negative'].append(true_negative)
lsh_stats['False Positive'].append(false_positive)
lsh_stats['False Negative'].append(false_negative)
lsh_stats['Candidate Pair count'].append(len(lsh.candidate_pairs))

##############################################################
Doc1 is Transcript: President Obama on What Books Mean to Him - The New York Times
Doc2 is Obama’s Secret to Surviving the White House Years: Books - The New York Times
Similarity is 65.41832669322709%
##############################################################
Doc1 is Cyberwar for Sale - The New York Times
Doc2 is The Perfect Weapon: How Russian Cyberpower Invaded the U.S. - The New York Times
Similarity is 65.01605995717344%
##############################################################
Doc1 is President Obama’s Farewell Address: Full Video and Text - The New York Times
Doc2 is Jolted by Deaths, Obama Found His Voice on Race - The New York Times
Similarity is 63.88246946186861%

            true positive count = 14
            true negative count = 107149
            false positive count = 17582
            false negative count = 5
        


### r = 5, b = 20, hash count = 100

In [17]:
lsh = LSH(b=20, r=5, hash_cnt=100)
similar_docs, sims = lsh.get_similar_documents(0.6)
similar_docs.sort(key=lambda x: -x[2])
print_docs(similar_docs[:3])
true_positive, true_negative, false_positive, false_negative = lsh.process_candidate_pairs(0.6)
lsh_stats['b'].append(20)
lsh_stats['r'].append(5)
lsh_stats['Number of hash functions'].append(100)
lsh_stats['True Positive'].append(true_positive)
lsh_stats['True Negative'].append(true_negative)
lsh_stats['False Positive'].append(false_positive)
lsh_stats['False Negative'].append(false_negative)
lsh_stats['Candidate Pair count'].append(len(lsh.candidate_pairs))


##############################################################
Doc1 is Transcript: President Obama on What Books Mean to Him - The New York Times
Doc2 is Obama’s Secret to Surviving the White House Years: Books - The New York Times
Similarity is 65.41832669322709%
##############################################################
Doc1 is Cyberwar for Sale - The New York Times
Doc2 is The Perfect Weapon: How Russian Cyberpower Invaded the U.S. - The New York Times
Similarity is 65.01605995717344%
##############################################################
Doc1 is President Obama’s Farewell Address: Full Video and Text - The New York Times
Doc2 is Jolted by Deaths, Obama Found His Voice on Race - The New York Times
Similarity is 63.88246946186861%

            true positive count = 18
            true negative count = 82662
            false positive count = 42069
            false negative count = 1
        


### r = 5, b = 40, hash count = 200

In [18]:
lsh = LSH(b=40, r=5, hash_cnt=200)
similar_docs, sims = lsh.get_similar_documents(0.6)
similar_docs.sort(key=lambda x: -x[2])
print_docs(similar_docs[:3])
true_positive, true_negative, false_positive, false_negative = lsh.process_candidate_pairs(0.6)
lsh_stats['b'].append(40)
lsh_stats['r'].append(5)
lsh_stats['Number of hash functions'].append(200)
lsh_stats['True Positive'].append(true_positive)
lsh_stats['True Negative'].append(true_negative)
lsh_stats['False Positive'].append(false_positive)
lsh_stats['False Negative'].append(false_negative)
lsh_stats['Candidate Pair count'].append(len(lsh.candidate_pairs))


##############################################################
Doc1 is Transcript: President Obama on What Books Mean to Him - The New York Times
Doc2 is Obama’s Secret to Surviving the White House Years: Books - The New York Times
Similarity is 65.41832669322709%
##############################################################
Doc1 is Cyberwar for Sale - The New York Times
Doc2 is The Perfect Weapon: How Russian Cyberpower Invaded the U.S. - The New York Times
Similarity is 65.01605995717344%
##############################################################
Doc1 is President Obama’s Farewell Address: Full Video and Text - The New York Times
Doc2 is Jolted by Deaths, Obama Found His Voice on Race - The New York Times
Similarity is 63.88246946186861%

            true positive count = 18
            true negative count = 58597
            false positive count = 66134
            false negative count = 1
        


### r = 10, b = 20, hash count = 200

In [19]:
lsh = LSH(b=20, r=10, hash_cnt=200)
similar_docs, sims = lsh.get_similar_documents(0.6)
similar_docs.sort(key=lambda x: -x[2])
print_docs(similar_docs[:3])
true_positive, true_negative, false_positive, false_negative = lsh.process_candidate_pairs(0.6)
lsh_stats['b'].append(20)
lsh_stats['r'].append(10)
lsh_stats['Number of hash functions'].append(200)
lsh_stats['True Positive'].append(true_positive)
lsh_stats['True Negative'].append(true_negative)
lsh_stats['False Positive'].append(false_positive)
lsh_stats['False Negative'].append(false_negative)
lsh_stats['Candidate Pair count'].append(len(lsh.candidate_pairs))

##############################################################
Doc1 is Transcript: President Obama on What Books Mean to Him - The New York Times
Doc2 is Obama’s Secret to Surviving the White House Years: Books - The New York Times
Similarity is 65.41832669322709%
##############################################################
Doc1 is Cyberwar for Sale - The New York Times
Doc2 is The Perfect Weapon: How Russian Cyberpower Invaded the U.S. - The New York Times
Similarity is 65.01605995717344%
##############################################################
Doc1 is President Obama’s Farewell Address: Full Video and Text - The New York Times
Doc2 is Jolted by Deaths, Obama Found His Voice on Race - The New York Times
Similarity is 63.88246946186861%

            true positive count = 1
            true negative count = 124157
            false positive count = 574
            false negative count = 18
        


In [20]:
pd.DataFrame(lsh_stats)

,b,r,Number of hash functions,True Positive,True Negative,False Positive,False Negative,Candidate Pair count
0,10,5,50,14,107149,17582,5,17596
1,20,5,100,18,82662,42069,1,42087
2,40,5,200,18,58597,66134,1,66152
3,20,10,200,1,124157,574,18,575
